<a href="https://colab.research.google.com/github/Dinhthixuanbinh/Note/blob/main/recognize_anything_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install timm
!git clone https://github.com/xinyu1205/recognize-anything.git
!pip install timm transformers fairscale pycocoevalcap
%cd recognize-anything

# Add this import to ram/models/utils.py

Cloning into 'recognize-anything'...
remote: Enumerating objects: 737, done.
remote: Counting objects: 100% (447/447), done.
remote: Compressing objects: 100% (224/224), done.
remote: Total 737 (delta 310), reused 285 (delta 223), pack-reused 290 (from 1)
Receiving objects: 100% (737/737), 27.14 MiB | 20.06 MiB/s, done.
Resolving deltas: 100% (397/397), done.
/content/recognize-anything/recognize-anything


In [ ]:
!git clone https://github.com/xinyu1205/recognize-anything.git
!pip install timm transformers fairscale pycocoevalcap
%cd recognize-anything

import os
import torch
import numpy as np
from PIL import Image
from ram.models import ram
from ram import get_transform

# Check and download pretrained weights
def download_checkpoints(model):
    print('You selected', model)
    if not os.path.exists('pretrained'):
        os.makedirs('pretrained')

    if model == "RAM":
        ram_weights_path = 'pretrained/ram_swin_large_14m.pth'
        if not os.path.exists(ram_weights_path):
            !wget https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth -O pretrained/ram_swin_large_14m.pth
        else:
            print("RAM weights already downloaded!")

# Download RAM model weights
model_name = "RAM"
download_checkpoints(model_name)

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Function for forward pass with RAM model
@torch.no_grad()
def forward_ram(model, imgs):
    image_embeds = model.image_proj(model.visual_encoder(imgs.to(device)))
    image_atts = torch.ones(image_embeds.size()[:-1], dtype=torch.long).to(device)

    label_embed = torch.nn.functional.relu(model.wordvec_proj(model.label_embed)).unsqueeze(0).repeat(imgs.shape[0], 1, 1)
    tagging_embed, _ = model.tagging_head(
        encoder_embeds=label_embed,
        encoder_hidden_states=image_embeds,
        encoder_attention_mask=image_atts,
        return_dict=False,
        mode='tagging',
    )

    logits = torch.sigmoid(model.fc(tagging_embed).squeeze(-1))
    targets = torch.where(
        logits > model.class_threshold.to(device),
        torch.tensor(1.0).to(device),
        torch.zeros(model.num_class).to(device)
    )

    tag_outputs, tag_logits = [], []
    for b in range(imgs.shape[0]):
        index = torch.nonzero(targets[b] == 1).squeeze(1)
        tokens = [model.tag_list[i] for i in index]
        scores = logits[b][index]
        tag_outputs.append([token.replace(" ", "_") for token in tokens])
        tag_logits.append(scores.cpu().numpy())

    return tag_outputs, tag_logits

# Load the RAM model
transform = get_transform(image_size=384)
model = ram(pretrained='pretrained/ram_swin_large_14m.pth', image_size=384, vit='swin_l')
model.eval()
model = model.to(device)




Cloning into 'recognize-anything'...
remote: Enumerating objects: 737, done.
remote: Counting objects: 100% (447/447), done.
remote: Compressing objects: 100% (224/224), done.
remote: Total 737 (delta 310), reused 285 (delta 223), pack-reused 290 (from 1)
Receiving objects: 100% (737/737), 27.14 MiB | 20.94 MiB/s, done.
Resolving deltas: 100% (397/397), done.
/content/recognize-anything/recognize-anything
You selected RAM
--2024-08-21 06:22:02--  https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth
Resolving huggingface.co (huggingface.co)... 18.239.50.16, 18.239.50.103, 18.239.50.49, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.16|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /spaces/xinyu1205/recognize-anything/resolve/main/ram_swin_large_14m.pth [following]
--2024-08-21 06:22:02--  https://huggingface.co/spaces/xinyu1205/recognize-anything/resolve/main/ram_swin_large_14m.p

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/encoder/layer/0/crossattention/self/query is tied
/encoder/layer/0/crossattention/self/key is tied
/encoder/layer/0/crossattention/self/value is tied
/encoder/layer/0/crossattention/output/dense is tied
/encoder/layer/0/crossattention/output/LayerNorm is tied
/encoder/layer/0/intermediate/dense is tied
/encoder/layer/0/output/dense is tied
/encoder/layer/0/output/LayerNorm is tied
/encoder/layer/1/crossattention/self/query is tied
/encoder/layer/1/crossattention/self/key is tied
/encoder/layer/1/crossattention/self/value is tied
/encoder/layer/1/crossattention/output/dense is tied
/encoder/layer/1/crossattention/output/LayerNorm is tied
/encoder/layer/1/intermediate/dense is tied
/encoder/layer/1/output/dense is tied
/encoder/layer/1/output/LayerNorm is tied
--------------
pretrained/ram_swin_large_14m.pth
--------------
load checkpoint from pretrained/ram_swin_large_14m.pth
vit: swin_l


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_your_image.jpg'